<img src=https://raw.githubusercontent.com/autonomio/hyperio/master/logo.png>

## How to recover best model from experiment log?
Due to system error or other reason where scan_object is no longer available, it's still possible to get best model/s using nothing but the experiment log. In the below notebook you will learn exactly how.



In [1]:
!pip install talos

     |████████████████████████████████| 26.6MB 28.6MB/s 
     |████████████████████████████████| 839kB 43.0MB/s 
  Created wheel for talos: filename=talos-0.6.3-cp36-none-any.whl size=49626 sha256=88fd2ba160682c65d0788881e37bb0f85c53085b3833ab2c5a194bb985ed316b
  Stored in directory: /root/.cache/pip/wheels/bb/d7/6b/86fd8b1fc7cfbd2c54796412f86efb5fb6a3a5c734014f6a66
  Created wheel for wrangle: filename=wrangle-0.6.7-cp36-none-any.whl size=49894 sha256=82c1ab2697753566da923816dad3365c8e634a863fcd3d405a913b6d2dff11ef
  Stored in directory: /root/.cache/pip/wheels/bf/1b/50/d0403ce6ef269e364894da7b50db68db14c4ac62c577561e2d
  Created wheel for astetik: filename=astetik-1.9.9-cp36-none-any.whl size=56960 sha256=de2a05979d2a4e75bc74e68b9e06d19a9224598e50b4e2c741b5952a3378040f
  Stored in directory: /root/.cache/pip/wheels/ae/70/21/c475cd079ec401dd6e1b9b1d42b4c38554ce12679bfb214aad
  Created wheel for chances: filename=chances-0.1.9-cp36-none-any.whl size=41609 sha256=27887d47ea2ef1471afcfd2

In [2]:
import keras
keras.__version__
import os

Using TensorFlow backend.


In [0]:
import sys
sys.path.insert(0, './talos/')

import sys
sys.path.insert(0, './wrangle/')

import talos
import wrangle
from keras.models import Sequential
from keras.layers import Dense

First we'll have to perform the `Scan()` experiment to produce the experiment log. Because the experiment log is stored on local machine, interrupted `Scan()` or other reason will not affect its availability. The experiment log is updated after each permutation; it contains an up-to-date record of the experiment.

In [8]:
# load the data
x, y = talos.templates.datasets.iris()
x_train, y_train, x_val, y_val = wrangle.array_split(x, y, .3)

# set the parameter space boundary
p = {'activation':['relu', 'elu'],
     'optimizer': ['Nadam', 'Adam'],
     'losses': ['logcosh'],
     'shapes': ['brick'],
     'first_neuron': [16, 32, 64, 128],
     'hidden_layers':[0, 1, 2, 3],
     'dropout': [.2, .3, .4],
     'batch_size': [20, 30, 40, 50],
     'epochs': [10]}

# define the input model
def iris_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=4, activation=params['activation']))

    talos.utils.hidden_layers(model, params, 3)

    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['acc'])

    out = model.fit(x_train, y_train, callbacks=[talos.utils.ExperimentLogCallback('minimal_iris', params)],
                     batch_size=params['batch_size'],
                     epochs=params['epochs'],
                     validation_data=[x_val, y_val],
                     verbose=0)

    return out, model

# start the experiment
scan_object = talos.Scan(x=x_train,
                         y=y_train,
                         x_val=x_val,
                         y_val=y_val,
                         model=iris_model,
                         experiment_name='reactivate',
                         params=p,
                         round_limit=10)



  0%|          | 0/10 [00:00<?, ?it/s]

`experiment_name` has to match `Scan(experiment_name)`


UnboundLocalError: ignored

Now we can assume the case where we no longer have access to the `scan_object`. In this `Scan(...experiment_name...)` was set to "reactivate" so we'll find a folder with that name in the present working directory. Next we have to find out what is the name of the experiment log.

In [0]:
# get the name of the experiment log
!ls -lhtr reactivate

In this case it will be the most recent one `092619223042.csv` so let's go ahead and recover the best models.

In [0]:
from talos.utils.recover_best_model import recover_best_model
results, models = recover_best_model(x_train=x_train,
                                     y_train=y_train,
                                     x_val=x_val,
                                     y_val=y_val,
                                     experiment_log='reactivate/092619221803.csv',
                                     input_model=iris_model,
                                     n_models=5,
                                     task='multi_label')

Now we can access the cross-validation results:

In [0]:
results

We can also access the models and make predictions with them:

In [0]:
models[0].predict(x_val)